# Отчёт по домашнему заданию

## Setup

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!cp /content/gdrive/My\ Drive/abbyy-nlp/train.jsonl /content
!cp /content/gdrive/My\ Drive/abbyy-nlp/dev.jsonl /content
!cp /content/gdrive/My\ Drive/abbyy-nlp/wiki.en.zip /content
!unzip wiki.en.zip
!rm wiki.en.zip
!rm wiki.en.vec

!pip install gensim
!pip install transformers
!pip install nltk
!pip install spacy

Archive:  wiki.en.zip
  inflating: wiki.en.vec             
  inflating: wiki.en.bin             
     |████████████████████████████████| 573kB 6.5MB/s 
     |████████████████████████████████| 890kB 35.5MB/s 
     |████████████████████████████████| 1.0MB 26.0MB/s 
     |████████████████████████████████| 3.7MB 39.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=2259704614dd711b30ae8d5df2795e8fdc9165cbc4fefca750b5d31af4e764ad
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


### Load dataset

In [0]:
from pprint import pprint
import json

with open("train.jsonl", ) as reader:
    train_dataset = list(map(json.loads, reader))

with open("dev.jsonl", ) as reader:
    dev_dataset = list(map(json.loads, reader))


In [4]:
from random import randint

n_samples = 2
print("Примеры из обучающей выборки: \n")
_ = [pprint(train_dataset[randint(0, len(train_dataset)-1)]) for _ in range(n_samples)]
print("\n", "="*50, "\n")
print("Примеры из тестовой выборки: \n")
_ = [pprint(dev_dataset[randint(0, len(dev_dataset)-1)]) for _ in range(n_samples)]

Примеры из обучающей выборки: 

{'answer': True,
 'passage': 'This is a list of all penalty shoot-outs that have occurred in '
            'the Finals tournament of the FIFA World Cup. Penalty shoot-outs '
            'were introduced as tie-breakers in the 1978 World Cup but did not '
            'occur before 1982. The first time a World Cup title was won by '
            'penalty shoot-out was in 1994. The only other time was in 2006. '
            'By the end of the 2018 edition, 30 shoot-outs have taken place in '
            'the World Cup. Of these, only two reached the sudden death stage '
            "after still being tied at the end of ``best of five kicks''.",
 'question': 'does the world cup final go to penalties',
 'title': 'List of FIFA World Cup penalty shoot-outs'}
{'answer': True,
 'passage': 'Clobetasol propionate /kloʊˈbeɪtəsɒl/ is a corticosteroid of the '
            'glucocorticoid class used to treat various skin disorders '
            'including eczema and pso

# Часть 1. [1 балл] Эксплоративный анализ

## 1.1 Доля yes и no классов в корпусе

In [5]:
get_yes_ratio = lambda ds: sum(map(lambda x: x['answer'], ds)) / len(ds)

print("Обучающая выборка")
print("\t yes:", get_yes_ratio(train_dataset))
print(f"\t no:", 1 - get_yes_ratio(train_dataset))

print("Тестовая выборка")
print(f"\t yes:", get_yes_ratio(dev_dataset))
print(f"\t no:", 1 - get_yes_ratio(dev_dataset))

Обучающая выборка
	 yes: 0.6231038506417736
	 no: 0.37689614935822635
Тестовая выборка
	 yes: 0.6217125382262997
	 no: 0.3782874617737003


## 1.2-3 Средняя длина вопроса и параграфа

In [6]:
from nltk.tokenize import TweetTokenizer

tokenizer = TweetTokenizer()
get_word_count = lambda paragraph, field: len(tokenizer.tokenize(paragraph[field]))
get_avg_word_count = lambda ds, field: sum(map(lambda x: get_word_count(x, field), ds)) / len(ds)
get_avg_char_count = lambda ds, field: sum(map(lambda x: len(x[field]), ds)) / len(ds)

print("Средняя длина вопроса:")
print("\t в обучающей выборке:")
print("\t\t word:", get_avg_word_count(train_dataset, 'question'))
print("\t\t char:", get_avg_char_count(train_dataset, 'question'))

print("\t в тестовой выборке:")
print("\t\t word:", get_avg_word_count(dev_dataset, 'question'))
print("\t\t char:", get_avg_char_count(dev_dataset, 'question'))

print("Средняя длина параграфа:")
print("\t в обучающей выборке:")
print("\t\t word:", get_avg_word_count(train_dataset, 'passage'))
print("\t\t char:", get_avg_char_count(train_dataset, 'passage'))

print("\t в тестовой выборке:")
print("\t\t word:", get_avg_word_count(dev_dataset, 'passage'))
print("\t\t char:", get_avg_char_count(dev_dataset, 'passage'))

Средняя длина вопроса:
	 в обучающей выборке:
		 word: 8.831971995332555
		 char: 43.99193805028111
	 в тестовой выборке:
		 word: 8.726299694189603
		 char: 43.206422018348626
Средняя длина параграфа:
	 в обучающей выборке:
		 word: 109.20685265726105
		 char: 565.6130264134931
	 в тестовой выборке:
		 word: 108.27033639143731
		 char: 559.0522935779817


## Эвристики, по которым был собран датасет

Авторы статьи оставляли только те вопросы, которые начинались с одного из слова {“did”, “do”, “does”, “is”, “are”, “was”, “were”, “have”, “has”, “can”, “could”, “will”, “would”}. После этого фильтровались вопросы, которые при запросе в поиске гугл, не выдавали страницу с википедией среди первых 5 выдач.

In [0]:
from collections import Counter

starter_tokens = {"did", "do", "does", "is", "are", "was", "were", "have", "has", "can", "could", "will", "would"}
first_question_words = Counter([doc['question'].split()[0] for doc in train_dataset])

### 20 наиболее часто встречающихся первых токенов вопроса

In [8]:
pprint(first_question_words.most_common(20))

[('is', 4190),
 ('can', 1136),
 ('does', 952),
 ('are', 693),
 ('do', 664),
 ('did', 461),
 ('was', 335),
 ('has', 302),
 ('will', 181),
 ('the', 91),
 ('have', 70),
 ('in', 35),
 ('were', 25),
 ('if', 17),
 ('a', 16),
 ('what', 11),
 ('when', 10),
 ('could', 9),
 ('who', 5),
 ('where', 5)]


### Процент вопросов, начинающихся со слов "did", "do", "does", "is", "are", "was", "were", "have", "has", "can", "could", "will", "would"

In [9]:
sum([first_question_words[word] for word in starter_tokens]) / len(train_dataset)

0.9570382942611647

### Примеры вопросов, начинающихся не на данные слова

In [10]:
from random import choices
indices = [i for i, x in enumerate([doc['question'].split()[0] not in starter_tokens for doc in train_dataset]) if x]
subsample = choices(indices, k=3)
_ = [pprint(train_dataset[i]) for i in subsample]

{'answer': False,
 'passage': 'Quantico has received positive reviews from critics, with praise '
            "for Chopra's performance and the diversity of the cast. However, "
            "the ``confusing dual timelines'' received some criticism. The "
            "series has been nominated for four People's Choice Awards, with "
            'Chopra winning two: Favorite Actress in a New TV Series in '
            "2016--making her the first South Asian to win a People's Choice "
            'Award--and Favorite Dramatic TV Actress in 2017. ABC renewed the '
            'series for a third season, which premiered on April 26, 2018. As '
            'part of the renewal process, Safran was replaced as primary '
            'showrunner by Michael Seitzman. On May 11, 2018, ABC cancelled '
            'the show after three seasons. The network will air the remaining '
            'episodes of the third season on Friday nights.',
 'question': 'what season of quantico is on tv now',
 'tit

# 2. Baseline
Для того, чтобы сравнивать качество определения ответов, необходимо сначала получить первое приближение. В качестве таких приближений использованы два классификатора: Dummy Classifier и FastText+LogReg.

## 2.1 Dummy Classifier
Dummy classifier представляет собой классификатор, который каждой новой паре вопрос-контекст предсказывает наиболее частый класс из обучающей выборки. При этом, если в качество алгоритма на тестовой выборке будет равно процентному соотношению самого частого класса в тестовой выборке, т.е. 0.62

In [0]:
train_questions = [doc['question'] for doc in train_dataset]
train_passages = [doc['passage'] for doc in train_dataset]
train_answers = [str(doc['answer']) for doc in train_dataset]

dev_questions = [doc['question'] for doc in dev_dataset]
dev_passages = [doc['passage'] for doc in dev_dataset]
dev_answers = [str(doc['answer']) for doc in dev_dataset]

In [12]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report

clf = DummyClassifier(strategy="most_frequent")
clf.fit(train_questions, train_answers)

DummyClassifier(constant=None, random_state=None, strategy='most_frequent')

In [13]:
dev_preds = clf.predict(dev_questions)
print(classification_report(dev_answers, dev_preds, digits=4))

              precision    recall  f1-score   support

       False     0.0000    0.0000    0.0000      1237
        True     0.6217    1.0000    0.7667      2033

    accuracy                         0.6217      3270
   macro avg     0.3109    0.5000    0.3834      3270
weighted avg     0.3865    0.6217    0.4767      3270



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## 2.2 Fasttext
В качестве более сложной модели, использующей признаки вопроса и контекста используется модель логистической регрессии, построенной на векторах FastText вопроса и контекста. Для токенов каждой последовательности были получены векторные представления, которые затем были осреднены. Полученные два вектора затем конкатенировались и подавались на вход классификатору.

#### Setup

In [14]:
from typing import List

# feature extractor
from gensim.models import FastText

# предобработка текстов
import nltk
from nltk.corpus import stopwords
import spacy

import numpy as np
# progress bar
from tqdm.auto import tqdm

nltk.download('stopwords')
model = FastText.load_fasttext_format("wiki.en.bin")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


#### Токенизация, лемматизация и векторизация токенов

In [15]:
stopwords_ = set(stopwords.words('english'))
nlp = spacy.load("en")

def tokenize(text: str) -> List[str]:
    """
    Функция, которая токенизирует и лемматизирует входной текст
    return: list of lemmas
    """
    return [word.lemma_ for word in nlp(text) if not word.is_punct]

def get_text_vector(tokens: List[str], model) -> np.ndarray:
    """
    Переводит последовательнось лемм в единый вектор
    """
    vectors = []
    for token in tokens:
        if token in model.wv:
            vectors.append(model.wv[token])
    # return np.mean(vectors, axis=0)
    return np.hstack([np.mean(vectors, axis=0), np.max(vectors, axis=0), np.std(vectors, axis=0)])

def preprocess_text(text: str) -> List[str]:
    """
    Предобработка текста. Включает в себя:
    1. приведение в нижний регистр
    2. токенизацию
    3. лемматизацию
    4. удаление стоп слов
    """
    text = text.lower()
    lemmas = tokenize(text)
    return [lemma for lemma in lemmas if lemma not in stopwords_]

def get_ds_vectors(dataset: List[dict], model) -> np.ndarray:
    """
    Функция, которая составляет векторное представление для каждой пары 
    (вопрос, контекст)
    """
    vectors = []
    for doc in tqdm(dataset):
        text = doc['question'] + " " + doc["passage"]
        lemmas = preprocess_text(text)
        vectors.append(get_text_vector(lemmas, model))

    vectors = np.vstack(vectors)

    return vectors

train_vectors = get_ds_vectors(train_dataset, model)
dev_vectors = get_ds_vectors(dev_dataset, model)

In [16]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter=500, solver="liblinear")
clf.fit(train_vectors, train_answers)
dev_preds = clf.predict(dev_vectors)

print(classification_report(dev_answers, dev_preds, digits=4))

              precision    recall  f1-score   support

       False     0.5119    0.2951    0.3744      1237
        True     0.6590    0.8288    0.7342      2033

    accuracy                         0.6269      3270
   macro avg     0.5854    0.5619    0.5543      3270
weighted avg     0.6033    0.6269    0.5981      3270



# Часть 3. Эмбеддинги предложений
В данном разделе предлагается использовать модель логистической регрессии, обученной на конкатенированных признаках, полученных с помощью BERT. Модель BERT для английского языка была взята из библиотеки Transformers.

In [17]:
from transformers import BertModel, BertTokenizer
from transformers import pipeline
import torch 

bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = BertModel.from_pretrained("bert-base-uncased")

feature_extraction = pipeline("feature-extraction", model=bert_model, tokenizer=bert_tokenizer, device=0)

В качестве векторного представления последовательности токенов было взято среднее векторных представлений токенов, а не вектор соответствующий служебному токену [CLS]. Данное решение было принято в ходе эксперимента, так как модель с классификатор, построенный на выходе токена [CLS] показала accuracy на 0.03 меньше. Векторы вопроса и контекста затем конкатенируются.

In [18]:
def extract_features(texts: List[str]) -> np.ndarray:
    """
    Составляет векторное представление для входной последовательности. 
    """
    features = []
    for text in tqdm(texts):
        features.append(np.asarray(feature_extraction(text)).mean(axis=1)[0, :])
    return np.vstack(features)

# Получим вектора для вопроса и контекста поотдельности
train_question_features = extract_features([d['question'] for d in train_dataset])
train_passage_features = extract_features([d['passage'] for d in train_dataset])

dev_question_features = extract_features([d['question'] for d in dev_dataset])
dev_passage_features = extract_features([d['passage'] for d in dev_dataset])

#### Train

In [20]:
# define classifier
clf = LogisticRegression(max_iter=10000, solver="liblinear")

# prepare train vetors
train_vectors = np.hstack((train_question_features, train_passage_features))

# fit logistic regression model
clf.fit(train_vectors, train_answers)

# validate on dev dataset
dev_vectors = np.hstack((dev_question_features, dev_passage_features))
dev_preds = clf.predict(dev_vectors)

# print out results
print(classification_report(dev_answers, dev_preds, digits=4))

              precision    recall  f1-score   support

       False     0.5702    0.4697    0.5151      1237
        True     0.7086    0.7846    0.7446      2033

    accuracy                         0.6654      3270
   macro avg     0.6394    0.6271    0.6299      3270
weighted avg     0.6562    0.6654    0.6578      3270



# Часть 4. DrQA подобная архитектура

Код для DrQA подобной архитектуры расположен в отдельном [репозитории](https://github.com/Muhamob/DrQA-1). Данный репозиторий является копией репозитория с уже реализованной архитектурой DrQA для задачи поиска начала и конца ответа. Части с чтением датасета, моделью и метриками была изменена под новый датасет.

### Архитектура

Для возможности использования архитектуры DrQA для задачи проверки истинности высказывания, можно в качестве одного из вариантов архитектуры использовать следующий:
1. убрать часть с предсказанием начала и конца ответа
2. Аггрегировать выходы BiLSTM кодировщика контекста (например, взять среднее или другие аггрегирующие функции)
3. Объединить признаки вопроса и контекста (конкатенировать)
4. Использовать классификатор на основе получившихся признаков (в данной работе используется линейный слой с 1 выходным признаком).

Также можно предлодить другие варианты архитектур. Например использовать векторы *similarity* для реккурентной нейронной сети. Затем, использовать выход последнего звена для предсказания ответа. 

### Результаты

Результаты работы для двух конфигураций следующие:
```yaml
- model:
    doc layers: 2
    question layers: 2
    hidden size: 64
    grad clipping: 3
  best_accuracy: 0.6889
- model:
    doc layers: 3
    question layers: 3
    hidden size: 64
    grad clipping: 3
  best_accuracy: 0.6923 
```

### Недостатки и возможные пути улучшения

Датасет для данной задачи является недостаточным, так что при обучении данных моделей, возможно, возникало переобучение. 

Недочёты моей реализации модели:
1. Отсутствие кросс-валидации. Валидация происходит только на одной отложенной выборке
2. Отсутствие подбора гиперпараметров

Возможности для улучшения качества модели:
1. Использование предобученных моделей (например, на датасете SQuAD и др.)
2. Подбор гиперпараметров
3. Аугментация данных

Гипотезы, которые хотелось бы проверить:
1. Использование BERT для извлечения признаков токенов
2. Использование другой архитектуры (предложена в разделе Архитектура)